# Assignment-1 ( 15 Marks)

## GNN Group 49

## Group Member Names:
1. Mukund Kumar (2023AA05458)
2. Basant Singh Bhaskar (2023aa05512)
3. Michael Joshua Glenn Marilyn (2023aa05394)
4. Nagamalla Nitin Kumar (2023ab05106)
5. Omkar Raju Iyer (2023aa05318)



## Part-1  ( 2 marks)

Use the given dataset to design GNN based model for Node Classification as per details given below:

Graph: The ogbn-products dataset is an undirected and unweighted graph, representing an Amazon product co-purchasing network [1]. Nodes represent products sold in Amazon, and edges between two products indicate that the products are purchased together. We follow [2] to process node features and target categories. Specifically, node features are generated by extracting bag-of-words features from the product descriptions followed by a Principal Component Analysis to reduce the dimension to 100.

URL: https://ogb.stanford.edu/docs/nodeprop/#ogbn-productsLinks to an external site.

You are expected to create Model using Pytorch Geometric MP-GNN based library.

Do not copy existing model given in OGB site but create your own model.


## Part-2 ( 1 Marks)

Use https://ogb.stanford.edu/docs/home/Links to an external site. to learn dataset loading and checking performance method.


## Part-3 ( 2X6 Marks)

Based  on loaded dataset also compute following point:

Diameter , number of nodes and edges , Global Clustering Coefficient of existing graph
Plot the graph with label
 Refer Relevant material from Book related to Subgraph generation and provide explanation how you are generating subgraph
Generate Node Induced Subgraph.
Generate Node embedding using 2-hop method for all nodes in subgraph using MP-GNN library in PyTorch Geometric
Plot Subgrpah and compute their Diameter.
 

# Instruction for Student:

Student is expected to use BITS Provided Labs and write Python code for model development.
Pytorch Library and Pytorch Geometric Library can be used
Python Library can be use
NetworkX Library needs to use.
This group assignment, so each member of group is expected to contribute evenly in completing the assignment.
Assignment should be submitted in PDF format which contains Codes and their outcomes along with explanation.
Link of Python code in BITS library needs to be given as we will be running the code in Lab environment.
Each group has to work independently should not copy code or outcome of other group. If Plagiarism found the that will be dealt as per BITS Policy.


----------------------------------------------------------------------------------------

## Part-1  ( 2 marks)

Use the given dataset to design GNN based model for Node Classification as per details given below:

Graph: The **ogbn-products** dataset is an undirected and unweighted graph, representing an Amazon product co-purchasing network **[1]**. 

**Nodes represent products sold** in Amazon, and **edges between two products** indicate that the **products are purchased together**. 

We follow **[2]** to process node features and target categories. Specifically, node features are generated by extracting bag-of-words features from the product descriptions followed by a Principal Component Analysis to reduce the dimension to 100.
 

URL: https://ogb.stanford.edu/docs/nodeprop/#ogbn-productsLinks to an external site.

You are expected to create Model using Pytorch Geometric MP-GNN based library.

Do not copy existing model given in OGB site but create your own model.

### References
[1] http://manikvarma.org/downloads/XC/XMLRepository.html

[2] Wei-Lin Chiang, Xuanqing Liu, Si Si, Yang Li, Samy Bengio, and Cho-Jui Hsieh. Cluster-GCN: An efficient algorithm for training deep and large graph convolutional networks. In ACM SIGKDD Conference on Knowledge Discovery and Data Mining (KDD), pp. 257–266, 2019.




In [ ]:
%env NX_CURGAPH_AUTOCONFIG=True
import torch
import torch_geometric as tgeo
from torch_geometric.data import DataLoader
from ogb.nodeproppred import PygNodePropPredDataset
from ogb.graphproppred import Evaluator
import networkx as nx
import matplotlib.pyplot as plt
import random
# import cugraph as nx

# SEED
SEED = 42
random.seed(SEED)
tgeo.seed_everything(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

# Constants
DEVICE = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
print(f"Device : {DEVICE}")

#!python -c "import ogb; print(ogb.__version__)"

In [ ]:
# Helper functions
def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()


def visualize_embedding(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    h = h.detach().cpu().numpy()
    plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    plt.show()

## Dataset format:

The data files for all the datasets are in the following sparse representation format:

Header Line: **Total_Points Num_Features Num_Labels**

1 line per datapoint : **label1,label2,...labelk ft1:ft1_val ft2:ft2_val ft3:ft3_val .. ftd:ftd_val**

For the small scale datasets, we have provided the complete data in one file. We have provided separate files for the train and test splits which contain the indices of the points that are in the train set and the test set. Each corresponding column of the split files contains a separate split.

For the large scale datasets, we have provided a single train and test split individually in two separate files.

In [ ]:
dataset_name = "ogbn-arxiv"

dataset = PygNodePropPredDataset(name=dataset_name)
pyg_graph_data = dataset[0]


print("Number of nodes: ", pyg_graph_data.num_nodes) # type: ignore
print("Number of edges: ", pyg_graph_data.num_edges) # type: ignore
print("Number of features per node: ", pyg_graph_data.num_features)
print("Number of classes: ", dataset.num_classes)

split_idx = dataset.get_idx_split()
# train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

# train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True) # type: ignore
# validation_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=True) # type: ignore
# test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=True) # type: ignore





In [ ]:
pyg_graph_data.train_mask

In [ ]:
# Define GNN Model
class GCN(torch.nn.Module):

    def __init__(self, in_channel, hidden_channel, out_channel):
        super().__init__()
        torch.manual_seed(SEED)
        self.conv1 = tgeo.nn.GCNConv(in_channel, hidden_channel) # type: ignore
        self.conv2 = tgeo.nn.GCNConv(hidden_channel, hidden_channel) # type: ignore
        self.classifier = tgeo.nn.Linear(hidden_channel, out_channel) # type: ignore

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = torch.nn.functional.tanh(h)
        h = self.conv2(h, edge_index)
        h = torch.nn.functional.tanh(h)  # Final GNN embedding space.

        # Apply a final (linear) classifier.
        out = self.classifier(h)

        return out, h

model = GCN(dataset.num_features, 64, dataset.num_classes)
print(model)

criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Define optimizer.

In [ ]:
# PCA on data

In [ ]:
def train(data):
    optimizer.zero_grad()  # Clear gradients.
    out, h = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss, h

for epoch in range(401):
    loss, h = train(data)
    if epoch % 10 == 0:
        visualize_embedding(h, color=data.y, epoch=epoch, loss=loss)
        time.sleep(0.3)

## Part-2 ( 1 Marks)

Use https://ogb.stanford.edu/docs/home/Links to an external site. to learn dataset loading and checking performance method.

In [ ]:
evaluator = Evaluator(name = dataset_name)
# You can learn the input and output format specification of the evaluator as follows.
# print(evaluator.expected_input_format) 
# print(evaluator.expected_output_format) 
input_dict = {"y_true": pyg_graph_data.y, "y_pred": y_pred}
result_dict = evaluator.eval(input_dict)


## Part-3 ( 2X6 Marks)

Based  on loaded dataset also compute following point:

1. Diameter , 
2. number of nodes and edges , 
3. Global Clustering Coefficient of existing graph

4. Plot the graph with label
    Refer Relevant material from Book related to Subgraph generation and provide explanation how you are generating subgraph

5. Generate Node-Induced Subgraph.
6. Generate Node embedding using 2-hop method for all nodes in subgraph using MP-GNN library in PyTorch Geometric
7. Plot Subgrpah and compute their Diameter.

In [ ]:

nx_graph = tgeo.utils.to_networkx(pyg_graph_data, to_undirected=False) # type: ignore

# (a) Diameter
max_diameter = 0
for component in nx.algorithms.components.connected_components(nx_graph):
    nx_subgraph = nx_graph.subgraph(component).copy()
    max_diameter = max(max_diameter, nx.algorithms.approximation.diameter(nx_subgraph))

print(f"Diameter of Graph : {max_diameter}")


# (b) Number of nodes and edges
num_nodes = nx_graph.number_of_nodes()
num_edges = nx_graph.number_of_edges()

print(f"Number of nodes : {num_nodes}") # type: ignore
print(f"Number of edges : {num_edges}") # type: ignore


# (c) Global Clustering Coefficient of existing graph
global_clustering_coefficient = nx.transitivity(nx_graph)
print(f"Global Clustering Coefficient : {global_clustering_coefficient}")

In [ ]:
# (d) Plot the Graph with Labels

plt.figure(figsize=(10, 10))
nx.draw(nx_graph, with_labels=True, node_size=10, font_size=8)
plt.title("Graph with Labels")
plt.show()


In [ ]:
# (e) Generate Node-Induced Subgraph

# Selecting 500 random nodes
nodes = 100
random_nodes = list(nx_graph.edges)[:nodes]
nx_subgraph = nx_graph.edge_subgraph(random_nodes)

# Plot the subgraph
plt.figure(figsize=(10, 10))
nx.draw(nx_subgraph, with_labels=True, node_size=100)
plt.title(f"Node-Induced Subgraph {nodes} Edges")
plt.show()

# Diameter of subgraph
max_diameter = 0
for component in nx.strongly_connected_components(nx_graph):
    nx_subgraph = nx_graph.subgraph(component).copy()
    max_diameter = max(max_diameter, nx.algorithms.approximation.diameter(nx_subgraph))

print(f"Diameter of subgraph : {max_diameter}")

In [ ]:
# (f) Generate Node Embeddings using 2-Hop method

class TwoHopGNN(torch.nn.Module):
    """2-hop method for GNN"""
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        torch.manual_seed(SEED)
        self.conv1 = tgeo.nn.GCNConv(in_channels, hidden_channels) # type: ignore
        self.conv2 = tgeo.nn.GCNConv(hidden_channels, hidden_channels) # type: ignore
        self.classifier = torch.nn.Linear(hidden_channels, out_features=out_channels)


    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = torch.nn.functional.tanh(h)
        h = self.conv2(h, edge_index)
        h = torch.nn.functional.tanh(h)

         # Apply a final (linear) classifier.
        out = self.classifier(h)

        return out, h
    

# Model
model = TwoHopGNN(dataset.num_features, 10, dataset.num_classes)
print(model)

_, h = model(pyg_graph_data.x, pyg_graph_data.edge_index) # type: ignore
print(f'Embedding shape: {list(h.shape)}')

visualize_embedding(h, color=pyg_graph_data.y) # type: ignore